# **AI Storytelling**

<img src="assets/logo.jpg" width="500" height="300">

This is Natural Language Processing platform that targets on conversion of short stories to audiobooks with features:

1.	Separated text into dialogues and narrations.
2.	Replaced pronouns by nouns in narrations.
3.	Extracted characters from narration
4.	Identified genders of characters using Gaussian NB with accuracy of 87%.
5.	Predicted age of characters using Random Forest Regressor with RMSE of 14.6.
6.	Finding emotions of characters from narrations with accuracy of x%.
7.	Performed audio processing for converting dialogues and narrations into audios.
8.	Combined audios to create the complete audiobook. 
9.	Final software development.


Based on the success of the project, it could be extended with image or video processing features in upcoming days.

### One-time Installations

In [ ]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_lg

### Import Libraries and Functions

In [1]:
# basic libraries
import re
import os
import string
import pandas as pd

In [2]:
# nlp libraries
import spacy
nlp = spacy.load("en_core_web_lg")

In [3]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# nlp libraries
import nltk
#nltk.download('wordnet')
#nltk.download("stopwords")
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

In [336]:
# emotion detection libraries
from transformers import pipeline

In [6]:
# text-to-speech libraries
from gtts import gTTS
from moviepy.editor import concatenate_audioclips, AudioFileClip
import librosa
import soundfile as sf

In [4]:
# other libraries
import joblib

### Files and Dataframes Defining

In [7]:
# checking stories
os.listdir("stories")

['beautiful garden.txt',
 'beneath the tides.txt',
 'cafe at midnight.txt',
 'the painted door.txt',
 'threads of imagination.txt']

In [277]:
# input title
title = input("Enter text file name:")

Enter text file name: threads of imagination


In [278]:
# opening text file
story = f"stories/{title}.txt"
with open(story, "r") as f:
    text = f.read()

In [279]:
# create dataframes
df_characters = pd.DataFrame(columns=["cid", "name", "num_dialogue", "num_occurrence"])
df_lines = pd.DataFrame(columns=["pid", "ndid", "character", "dialogue", "narration"])

# 1. Lines Identification

In [280]:
# converting text to paragraphs
paragraphs = text.split("\n")
non_empty_paragraphs = list(filter(lambda x: x != '', paragraphs))

In [281]:
# function to identify narrations and dialogues
def identify_narrations_and_dialogues(paragraph):
    """
    :param paragraph: string of paragraph in a story
    :return: list of tuples in (id, name_of_speaker, dialogue, narration) format
    """
    divisions = paragraph.split('"')
    divisions = list(filter(lambda x: x != '', divisions))
    i, spoken = 0, False
    narrations_and_dialogues = []
    for division in divisions:
        start_index = paragraph.find(division)
        end_index = start_index + len(division) - 1
        try:
            if '"'==paragraph[start_index-1:start_index] and '"'==paragraph[end_index+1:end_index+2] and spoken==False:
                # dialogues
                narrations_and_dialogues.append((i, None, division, None))
                spoken = True
            else:
                # narrations
                narrations_and_dialogues.append((i, None, None, division))
                spoken = False
        except:
            # narrations
            narrations_and_dialogues.append((i, None, None, division))
            spoken = False
        i += 1
    return narrations_and_dialogues


In [282]:
# identifying lines (narrations or dialogues) from each paragraphs
pid_num = 0
for paragraph in non_empty_paragraphs:
    for row in [(pid_num,)+nad for nad in identify_narrations_and_dialogues(paragraph)]:
        df_lines = df_lines._append(pd.Series(row, index=df_lines.columns), ignore_index=True)
    pid_num += 1

In [283]:
# set index using paragraph and nd
df_lines.set_index(['pid', 'ndid'], inplace=True)

In [284]:
# narrations and dialogues
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1         None                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1         None                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1         None                 What's the story behind this book?   
3   0         None                                               None   
    1         None  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Deep in the heart of a sprawling city, where s...  
    1                                                  None  
    2      was a haven for those seeking refuge from the...  
1   0     One rainy afternoon, as the sound of raindrops...  
    1                                                  None  
2   0                  Intrigued, Oliver approached Clara.   
    1                                                  None  
3   0                Clara, a twinkle in her eye, replied,   
    1                                                  None  
4   0     Oliver, fueled by curiosity, decided to purcha...

# 2. Characters Identification

In [285]:
# function to find subjects speaking corresponding dialogues
def identify_subjects(sentence):
    """
    :param sentence: string of sentence in the story
    :return: individual speaker
    """
    tokens = []
    individual, individuals = '', []
    
    for tok in reversed(nlp(sentence)):
        if str(tok)=='.' and tok.dep_ == 'punct':
            break
        tokens.append([tok, tok.dep_])
    tokens = tokens[::-1]
    
    for i in range(len(tokens)-1, -1, -1):
        if tokens[i][1] == "nsubj":
            individual = str(tokens[i][0])
        elif (tokens[i][1] in ['compound', 'det'] and individual != ''):
            individual = str(tokens[i][0]) + " " + individual
        else:
            individuals.append(individual)
            
    individuals.append(individual)
    return list(set([ind for ind in individuals if ind != '']))

In [286]:
# finding semantic similarity
def calculating_semantic_similarity(subject, sentence):
    """
    :param subject: list of strings available in a sentence
    :return: similarity score
    """
    subject_doc = nlp(subject)
    sentence_doc = nlp(sentence)
    
    subject_vec = subject_doc.vector
    sentence_vec = sentence_doc.vector
    
    subject_vec_norm = subject_doc.vector_norm
    sentence_vec_norm = sentence_doc.vector_norm
    
    if subject_vec_norm > 0 and sentence_vec_norm > 0:
        similarity_score = subject_vec.dot(sentence_vec) / (subject_vec_norm * sentence_vec_norm)
    else:
        similarity_score = 0
    
    return similarity_score

In [287]:
# finding subjects from dialogues and narrations
dlg_list = df_lines.index[df_lines['dialogue'].notna()].tolist()

for dlgi in dlg_list:
    if df_lines.loc[(dlgi[0], dlgi[1]), 'dialogue'][-1] == ',':
        full_narration = df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration']
        final_narration = full_narration.rstrip().rstrip(string.punctuation)
        subjects = identify_subjects(final_narration)
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]+1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k]) if relevant_dictionary else None
    else:
        full_narration = df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration']
        final_narration = full_narration.rstrip().rstrip(string.punctuation)
        subjects = identify_subjects(final_narration)
        relevant_dictionary = dict((subject.lower(), calculating_semantic_similarity(subject.lower(), df_lines.loc[(dlgi[0], dlgi[1]-1), 'narration'].lower())) for subject in subjects)
        relevant_subject = max(relevant_dictionary, key=lambda k: relevant_dictionary[k]) if relevant_dictionary else None
    df_lines.loc[(dlgi[0], dlgi[1]), 'character'] = relevant_subject

In [288]:
# dialogues and corresponding characters
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        cover                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  
pid ndid                                                     
0   0     Deep in the heart of a sprawling city, where s...  
    1                                                  None  
    2      was a haven for those seeking refuge from the...  
1   0     One rainy afternoon, as the sound of raindrops...  
    1                                                  None  
2   0                  Intrigued, Oliver approached Clara.   
    1                                                  None  
3   0                Clara, a twinkle in her eye, replied,   
    1                                                  None  
4   0     Oliver, fueled by curiosity, decided to purcha...

In [289]:
# check characters
df_lines.character.unique()

array([None, 'clara', 'cover', 'oliver', 'the book', 'seraphina',
       'a that'], dtype=object)

In [290]:
# gathering characters from df_lines
characters = df_lines['character'].value_counts()
df_characters['name'] = characters.index
df_characters['num_dialogue'] = characters.values

In [291]:
# declaring cid as index
df_characters['cid'] = range(0, len(df_characters))
df_characters.set_index('cid', inplace=True)

In [292]:
# counting number of times these names occurred in text
df_characters['num_occurrence'] = df_characters['name'].apply(lambda x: len(re.findall(x, text.lower())))

In [293]:
# top 10 characters
df_characters.head(10)

,name,num_dialogue,num_occurrence
cid,,,
0,clara,2,8
1,cover,1,3
2,oliver,1,10
3,the book,1,3
4,seraphina,1,4
5,a that,1,0


# 3. Cleaned Dialogue Generation

In [294]:
# defining stopwords set and lemma
set_of_stopwords = set(stopwords.words("english"))
lemma = WordNetLemmatizer()

In [295]:
# function to remove contractions and replace by spaces
def remove_contractions(text):

    contractions = ["ain't", "aren't", "can't", "can't've", "'cause", "could've", "couldn't", "couldn't've", "didn't", "doesn't", "don't",
                    "hadn't", "hadn't've", "hasn't", "haven't", "he'd", "he'd've", "he'll", "he'll've", "he's", "how'd", "how'd'y", "how'll", "how's",
                    "I'd", "I'd've", "I'll", "I'll've", "I'm", "I've", "isn't", "it'd", "it'd've", "it'll", "it'll've", "it's", "let's",
                    "ma'am", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've",
                    "o'clock", "oughtn't", "oughtn't've", "shan't", "sha'n't", "shan't've", "she'd", "she'd've", "she'll", "she'll've", "she's",
                    "should've", "shouldn't", "shouldn't've", "so've", "so's", "that'd", "that'd've", "that's", "there'd", "there'd've", "there's",
                    "they'd", "they'd've", "they'll", "they'll've", "they're", "they've", "to've", "wasn't",
                    "we'd", "we'd've", "we'll", "we'll've", "we're", "we've", "weren't", "what'll", "what'll've", "what're", "what's", "what've",
                    "when's", "when've", "where'd", "where's", "where've", "who'll", "who'll've", "who's", "who've", "why's", "why've",
                    "will've", "won't", "won't've", "would've", "wouldn't", "wouldn't've", "y'all", "y'all'd", "y'all'd've", "y'all're", "y'all've",
                    "you'd", "you'd've", "you'll", "you'll've", "you're", "you've", "gonna"]

    for contraction in contractions:
        text = text.replace(contraction, " ")

    return text

In [296]:
# function to remove punctuations
def remove_punctuations(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [297]:
# function to check noun
def is_noun(word):
    synsets = wordnet.synsets(word)
    for synset in synsets:
        if synset.pos() == 'n':
            return True
    return False

In [298]:
# dialogue to cleaned dialogue
def cleaned_dialogue(text):
    if text is None:
        return ''
    contractionless_text = remove_contractions(text)
    punctuationless_text = remove_punctuations(contractionless_text)
    tokens = nltk.word_tokenize(punctuationless_text)
    filtered_words = [token.lower() for token in tokens if token.lower() not in set_of_stopwords]
    lemmatized_words = [lemma.lemmatize(word) for word in filtered_words]
    words = [word for word in lemmatized_words if is_noun(word)]
    return ' '.join(words)

In [299]:
# list of words for dialogue
dialogue_list = []
for dialogue in df_lines.dialogue:
    dialogue_list.append(cleaned_dialogue(dialogue))
df_lines['cleaned_dialogue'] = dialogue_list

In [300]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        cover                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Deep in the heart of a sprawling city, where s...   
    1                                                  None   
    2      was a haven for those seeking refuge from the...   
1   0     One rainy afternoon, as the sound of raindrops...   
    1                                                  None   
2   0                  Intrigued, Oliver approached Clara.    
    1                                                  None   
3   0                Clara, a twinkle in her eye, replied,    
    1                                                  None   
4   0     Oliver, fueled by curiosity, decided to purcha...   

                                           cleaned_dialogue  
pid ndid                                                     
0   0                                                        
    1                                                  page  
    2                                                        
1   0                                                        
    1                                           atlas realm  
2   0                                                        
    1                                     story behind book  
3   0                                                        
    1     special one legend hold key realm imagination ...  
4   0

# 4. Age Detection

In [301]:
# loading age count vectorizer
loaded_age_cv = joblib.load('attributes/age/age_cv.joblib')

In [302]:
# creating sparse matrix for transformed values and array
age_sparse_matrix = loaded_age_cv.transform(df_lines['cleaned_dialogue'])
age_sparse_matrix_array = age_sparse_matrix.toarray()

In [303]:
# loading age model
loaded_age_model = joblib.load('attributes/age/age_model.joblib')

In [304]:
# defining X and predicting
df_lines['age'] = loaded_age_model.predict(age_sparse_matrix_array)

In [305]:
# define age as None if dialogue is None
df_lines['age'] = df_lines.apply(lambda row: None if pd.isna(row['dialogue']) else row['age'], axis=1)

In [306]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        cover                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Deep in the heart of a sprawling city, where s...   
    1                                                  None   
    2      was a haven for those seeking refuge from the...   
1   0     One rainy afternoon, as the sound of raindrops...   
    1                                                  None   
2   0                  Intrigued, Oliver approached Clara.    
    1                                                  None   
3   0                Clara, a twinkle in her eye, replied,    
    1                                                  None   
4   0     Oliver, fueled by curiosity, decided to purcha...   

                                           cleaned_dialogue   age  
pid ndid                                                           
0   0                                                         NaN  
    1                                                  page  5.18  
    2                                                         NaN  
1   0                                                         NaN  
    1                                           atlas realm  5.18  
2   0                                                         NaN  
    1                                     story behind book  9.04  
3   0                                                         NaN  
    1     special one legend hold key realm imagination ...  8.38  
4   0                                                         NaN

In [307]:
# calculating age of character
for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'age'] = df_lines[df_lines['character'] == character]['age'].mean()

In [308]:
# top 10 rows with age
df_characters.head(10)

,name,num_dialogue,num_occurrence,age
cid,,,,
0,clara,2,8,6.78
1,cover,1,3,5.18
2,oliver,1,10,9.04
3,the book,1,3,5.18
4,seraphina,1,4,5.33
5,a that,1,0,5.18


# 5. Gender Detection

In [309]:
# loading gender count vectorizer
loaded_gender_cv = joblib.load('attributes/gender/gender_cv.joblib')

In [310]:
# creating sparse matrix for transformed values and array
gender_sparse_matrix = loaded_gender_cv.transform(df_lines['cleaned_dialogue'])
gender_sparse_matrix_array = gender_sparse_matrix.toarray()

In [311]:
# loading gender model
loaded_gender_model = joblib.load('attributes/gender/gender_model.joblib')

In [312]:
# defining X and predicting
df_lines['gender'] = loaded_gender_model.predict(gender_sparse_matrix_array)

In [313]:
# define gender as None if dialogue is None
df_lines['gender'] = df_lines.apply(lambda row: None if pd.isna(row['dialogue']) else row['gender'], axis=1)

In [314]:
# define gender based upon he/she
df_lines['gender'] = df_lines['character'].apply(lambda row: 'Male' if row=='he' else 'Female' if row=='she' else row)

In [315]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        cover                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Deep in the heart of a sprawling city, where s...   
    1                                                  None   
    2      was a haven for those seeking refuge from the...   
1   0     One rainy afternoon, as the sound of raindrops...   
    1                                                  None   
2   0                  Intrigued, Oliver approached Clara.    
    1                                                  None   
3   0                Clara, a twinkle in her eye, replied,    
    1                                                  None   
4   0     Oliver, fueled by curiosity, decided to purcha...   

                                           cleaned_dialogue   age  gender  
pid ndid                                                                   
0   0                                                         NaN    None  
    1                                                  page  5.18   clara  
    2                                                         NaN    None  
1   0                                                         NaN    None  
    1                                           atlas realm  5.18   cover  
2   0                                                         NaN    None  
    1                                     story behind book  9.04  oliver  
3   0                                                         NaN    None  
    1     special one legend hold key realm imagination ...  8.38   clara  
4   0                                                         NaN    None

In [316]:
# defining empty columns
df_characters['male_count'], df_characters['female_count'] = None, None

In [317]:
# counting male and female values of character
for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'male_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Male")])
    df_characters.loc[df_characters['name'] == character, 'female_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Female")])

In [318]:
# defining gender column
df_characters['gender'] = df_characters.apply(lambda row: 'Female' if row['female_count'] > row['male_count'] else ('Male' if row['female_count'] < row['male_count'] else None), axis=1)

In [319]:
# top 10 rows with male and female counts
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,clara,2,8,6.78,0,0,None
1,cover,1,3,5.18,0,0,None
2,oliver,1,10,9.04,0,0,None
3,the book,1,3,5.18,0,0,None
4,seraphina,1,4,5.33,0,0,None
5,a that,1,0,5.18,0,0,None


# 6. He/She Replacement

In [320]:
# characters not pronouns
other_characters = df_characters[~df_characters['name'].isin(['he', 'she', 'that', 'they', 'it'])]['name']
other_characters = list(other_characters)
print(other_characters)

['clara', 'cover', 'oliver', 'the book', 'seraphina', 'a that']


In [321]:
# names in narration
def find_names_in_narration(narration):
    if narration is None:
        return None
    names = dict()
    for character in other_characters:
        location = narration.lower().rfind(character)
        if location != -1:
            names[character] = location
    if len(names) == 0:
        return None
    return names.keys()
df_lines['narrated_names'] = df_lines['narration'].apply(lambda row: find_names_in_narration(row))

In [322]:
# he she replacement by names
def he_she_replacement(i, gender):
    j = i-1
    last_row_narrated_names = df_lines.iloc[j]['narrated_names']
    while last_row_narrated_names is None:
        j -= 1
        last_row_narrated_names = df_lines.iloc[j]['narrated_names']
    last_row_narrated_names = list(last_row_narrated_names)
    for name in last_row_narrated_names:
        if list(df_characters[df_characters['name'] == name]['gender'])[0] == gender:
            return name
    return last_row_narrated_names[0]

In [323]:
# replacing all he/she in df_lines
for i in range(1, len(df_lines)):
    if df_lines.iloc[i, 0] not in other_characters+['he', 'she'] and df_lines.iloc[i, 0] is not None:
        df_lines.iloc[i, 0] = None
        df_lines.iloc[i, 4] = None
    elif df_lines.iloc[i, 0] == 'he':
        df_lines.iloc[i, 0] = he_she_replacement(i, 'Male')
    elif df_lines.iloc[i, 0] == 'she':
        df_lines.iloc[i, 0] = he_she_replacement(i, 'Female')
    else:
        pass

In [324]:
# lines with narrated names
df_lines.tail(10)

character                        dialogue  \
pid ndid                                             
6   2         None                            None   
7   0         None                            None   
8   0         None                            None   
9   0         None                            None   
10  0         None                            None   
    1         None                 Whimsical Pages   
    2         None                            None   
11  0         None                            None   
    1       a that  The Atlas of Uncharted Realms,   
    2         None                            None   

                                                  narration cleaned_dialogue  \
pid ndid                                                                       
6   2                                 Seraphina proclaimed.                    
7   0     Together, Oliver and Seraphina embarked on a q...                    
8   0     Back in the city, Clara sensed a connection to...                    
9   0     As Oliver and Seraphina faced trials and trium...                    
10  0     The once-quiet bookstore buzzed with energy as...                    
    1                                                  None             page   
    2      came to life, echoing with laughter, gasps, a...                    
11  0     As Oliver's journey unfolded, he discovered th...                    
    1                                                  None      atlas realm   
    2      a tapestry of interconnected stories unfolded...                    

           age  gender                        narrated_names  
pid ndid                                                      
6   2      NaN    None                           (seraphina)  
7   0      NaN    None                   (oliver, seraphina)  
8   0      NaN    None                        (clara, cover)  
9   0      NaN    None  (clara, oliver, the book, seraphina)  
10  0      NaN    None                       (clara, oliver)  
    1     5.18    None                                  None  
    2      NaN    None                                  None  
11  0      NaN    None                       (cover, oliver)  
    1     5.18  a that                                  None  
    2      NaN    None                                  None

In [325]:
# filtered df_characters with other_characters
df_characters = df_characters[df_characters['name'].isin(other_characters)]

In [326]:
# filtered df_characters only
df_characters.head()

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,clara,2,8,6.78,0,0,None
1,cover,1,3,5.18,0,0,None
2,oliver,1,10,9.04,0,0,None
3,the book,1,3,5.18,0,0,None
4,seraphina,1,4,5.33,0,0,None


In [327]:
# gathering characters from df_lines
df_lines_vc = df_lines['character'].value_counts()
filtered_df = pd.DataFrame(df_lines_vc.reset_index(), columns=['character', 'count'])

In [328]:
# filtered_df
filtered_df.head()

,character,count
0,clara,2
1,cover,1
2,oliver,1
3,the book,1
4,seraphina,1


In [329]:
# number of dialogues, male and female counts and gender after he/she replacement
df_characters['num_dialogue'] = df_characters['name'].apply(lambda x: list(filtered_df[filtered_df['character'] == x]['count'])[0])

for character in df_characters['name']:
    df_characters.loc[df_characters['name'] == character, 'male_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Male")])
    df_characters.loc[df_characters['name'] == character, 'female_count'] = len(df_lines[(df_lines['character'] == character) & (df_lines['gender'] == "Female")])

df_characters['gender'] = df_characters.apply(lambda row: 'Female' if row['female_count'] > row['male_count'] else ('Male' if row['female_count'] < row['male_count'] else None), axis=1)

In [330]:
# new df_characters with new number of dialogues, male count and female count
df_characters.head(10)

,name,num_dialogue,num_occurrence,age,male_count,female_count,gender
cid,,,,,,,
0,clara,2,8,6.78,0,0,None
1,cover,1,3,5.18,0,0,None
2,oliver,1,10,9.04,0,0,None
3,the book,1,3,5.18,0,0,None
4,seraphina,1,4,5.33,0,0,None
5,a that,1,0,5.18,0,0,None


# 7. Emotions Detection

In [337]:
# creating emotion detection classifier
ed_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [347]:
# function to return better dictionary
def show_emotions(dialogue):
    emotions_dict = dict()
    edc = ed_classifier(dialogue)
    for x in edc[0]:
        emotions_dict[x['label']] = x['score']
    return emotions_dict

In [363]:
# using ed_classifier for showing emotions for each dialogue
df_lines['anger'], df_lines['disgust'], df_lines['fear'], df_lines['joy'], df_lines['neutral'], df_lines['sadness'], df_lines['surprise'] = None, None, None, None, None, None, None
df_lines[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']] = df_lines['dialogue'].apply(lambda x: list(show_emotions(x).values()) if x is not None else [None, None, None, None, None, None, None]).tolist()

In [364]:
# top 10 rows
df_lines.head(10)

character                                           dialogue  \
pid ndid                                                                
0   0         None                                               None   
    1        clara                                   Whimsical Pages,   
    2         None                                               None   
1   0         None                                               None   
    1        cover                     The Atlas of Uncharted Realms.   
2   0         None                                               None   
    1       oliver                 What's the story behind this book?   
3   0         None                                               None   
    1        clara  Ah, that's a special one. Legend has it that i...   
4   0         None                                               None   

                                                  narration  \
pid ndid                                                      
0   0     Deep in the heart of a sprawling city, where s...   
    1                                                  None   
    2      was a haven for those seeking refuge from the...   
1   0     One rainy afternoon, as the sound of raindrops...   
    1                                                  None   
2   0                  Intrigued, Oliver approached Clara.    
    1                                                  None   
3   0                Clara, a twinkle in her eye, replied,    
    1                                                  None   
4   0     Oliver, fueled by curiosity, decided to purcha...   

                                           cleaned_dialogue   age  gender  \
pid ndid                                                                    
0   0                                                         NaN    None   
    1                                                  page  5.18   clara   
    2                                                         NaN    None   
1   0                                                         NaN    None   
    1                                           atlas realm  5.18   cover   
2   0                                                         NaN    None   
    1                                     story behind book  9.04  oliver   
3   0                                                         NaN    None   
    1     special one legend hold key realm imagination ...  8.38   clara   
4   0                                                         NaN    None   

              narrated_names     anger   disgust      fear       joy  \
pid ndid                                                               
0   0                (clara)       NaN       NaN       NaN       NaN   
    1                   None  0.005425  0.042973  0.071832  0.305740   
    2                (clara)       NaN       NaN       NaN       NaN   
1   0        (cover, oliver)       NaN       NaN       NaN       NaN   
    1                   None  0.014949  0.019127  0.018491  0.009880   
2   0        (clara, oliver)       NaN       NaN       NaN       NaN   
    1                   None  0.018343  0.005769  0.012591  0.005391   
3   0                (clara)       NaN       NaN       NaN       NaN   
    1                   None  0.042832  0.024846  0.013596  0.005710   
4   0     (oliver, the book)       NaN       NaN       NaN       NaN   

           neutral   sadness  surprise  
pid ndid                                
0   0          NaN       NaN       NaN  
    1     0.476073  0.050887  0.047069  
    2          NaN       NaN       NaN  
1   0          NaN       NaN       NaN  
    1     0.901857  0.020523  0.015172  
2   0          NaN       NaN       NaN  
    1     0.791414  0.004370  0.162122  
3   0          NaN       NaN       NaN  
    1     0.877496  0.005339  0.030180  
4   0          NaN       NaN       NaN

In [366]:
# check dialogues and corresponding emotions only
df_lines[~df_lines['dialogue'].isna()][['dialogue', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']].head(10)

,,dialogue,anger,disgust,fear,joy,neutral,sadness,surprise
pid,ndid,,,,,,,,
0,1,"Whimsical Pages,",0.005425,0.042973,0.071832,0.305740,0.476073,0.050887,0.047069
1,1,The Atlas of Uncharted Realms.,0.014949,0.019127,0.018491,0.009880,0.901857,0.020523,0.015172
2,1,What's the story behind this book?,0.018343,0.005769,0.012591,0.005391,0.791414,0.004370,0.162122
3,1,"Ah, that's a special one. Legend has it that i...",0.042832,0.024846,0.013596,0.005710,0.877496,0.005339,0.030180
5,1,"The Atlas of Uncharted Realms,",0.011839,0.010017,0.013543,0.010646,0.905701,0.023561,0.024693
6,1,"You, dear traveler, are destined to play a rol...",0.005183,0.005265,0.002864,0.051998,0.916293,0.005480,0.012917
10,1,Whimsical Pages,0.007544,0.056585,0.096496,0.363998,0.364428,0.064259,0.046689
11,1,"The Atlas of Uncharted Realms,",0.011839,0.010017,0.013543,0.010646,0.905701,0.023561,0.024693


# 8. Audio Generation

In [331]:
# creating audios
print("Step 1: CONVERSIONS")
i = 0
for index, row in df_lines.iterrows():
    if row['dialogue'] is not None:
        speech_gtts = gTTS(text=row['dialogue'], lang='en', slow=False, tld='co.in')
    else:
        speech_gtts = gTTS(text=row['narration'], lang='en', slow=False, tld='ie')
    temp_file = f"conversions/{i}.mp3"
    print(f"Line {i+1}/{df_lines.shape[0]} converted.")
    speech_gtts.save(temp_file)
    i += 1
print("Conversions finished.")

Step 1: CONVERSIONS
Line 1/25 converted.
Line 2/25 converted.
Line 3/25 converted.
Line 4/25 converted.
Line 5/25 converted.
Line 6/25 converted.
Line 7/25 converted.
Line 8/25 converted.
Line 9/25 converted.
Line 10/25 converted.
Line 11/25 converted.
Line 12/25 converted.
Line 13/25 converted.
Line 14/25 converted.
Line 15/25 converted.
Line 16/25 converted.
Line 17/25 converted.
Line 18/25 converted.
Line 19/25 converted.
Line 20/25 converted.
Line 21/25 converted.
Line 22/25 converted.
Line 23/25 converted.
Line 24/25 converted.
Line 25/25 converted.
Conversions finished.


In [332]:
# combining audios
print("Step 2: COMBINATION")
clips = [AudioFileClip(f"conversions/{i}.mp3") for i in range(df_lines.shape[0])]
final_clip = concatenate_audioclips(clips)
print("Combination finished.")

Step 2: COMBINATION
Combination finished.


In [333]:
# adjust speed
print("Step 3: ADJUSTMENT")
final_clip.write_audiofile("conversions/final_slow.mp3")
print("final_slow.mp3 downloaded.")
y, sr = librosa.load("conversions/final_slow.mp3", sr=None)
y_speed = librosa.effects.time_stretch(y, rate=1.25)
print("Speeded over.")

Step 3: ADJUSTMENT
MoviePy - Writing audio in conversions/final_slow.mp3


MoviePy - Done.
final_slow.mp3 downloaded.
Speeded over.


In [334]:
# removing audios
print("Step 4: REMOVAL")
for i in range(df_lines.shape[0]):
    os.remove(f"conversions/{i}.mp3")
    print(f"Removed {i}.mp3")
os.remove("conversions/final_slow.mp3")
print("Removal over.")

Step 4: REMOVAL
Removed 0.mp3
Removed 1.mp3
Removed 2.mp3
Removed 3.mp3
Removed 4.mp3
Removed 5.mp3
Removed 6.mp3
Removed 7.mp3
Removed 8.mp3
Removed 9.mp3
Removed 10.mp3
Removed 11.mp3
Removed 12.mp3
Removed 13.mp3
Removed 14.mp3
Removed 15.mp3
Removed 16.mp3
Removed 17.mp3
Removed 18.mp3
Removed 19.mp3
Removed 20.mp3
Removed 21.mp3
Removed 22.mp3
Removed 23.mp3
Removed 24.mp3
Removal over.


In [335]:
# downloading final audio
print("Step 5: DOWNLOADING")
final_title = title.replace(" ", "_")
sf.write(f"audiobooks/{final_title}.mp3", y_speed, sr)
print(f"{final_title}.mp3 downloaded")

Step 5: DOWNLOADING
threads_of_imagination.mp3 downloaded
